# Sentiment Analysis with Large Language Model embeddings

## Our Dataset

This dataset describes the contents of the heart-disease diagnosis.

The dataset in this study is from [Kaggle](https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/data), which is called Twitter US Airline Sentiment.

- Dataset: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment/data

## Variable Table

| Original Dataset             | Data Type     | Description    |                                                         
|------------------------------|---------------|---------------------------------------------------------------------------------------------|
| tweet_id                     | ID            | A unique identifier for each tweet.                                                         | 
| airline_sentiment            | Categorical   | The sentiment expressed in the tweet (positive, neutral, negative).                         | 
| airline_sentiment_confidence | Numerical     | Confidence score in the sentiment label (0 to 1).                                           | 
| negativereason               | Categorical   | Reason for negative sentiment (e.g., "Late Flight", "Customer Service Issue").              | 
| negativereason_confidence    | Numerical     | Confidence score in the negative reason label (0 to 1).                                     | 
| airline                      | Categorical   | The airline mentioned in the tweet (e.g., United, Delta, etc.).                             | 
| airline_sentiment_gold       | Categorical   | Sentiment label by trusted annotator (gold standard).                                       | 
| name                         | Text          | Name of the user who posted the tweet.                                                      | 
| negativereason_gold          | Categorical   | Negative reason label by trusted annotator (gold standard).                                 | 
| retweet_count                | Numerical     | Number of times the tweet was retweeted.                                                    | 
| text                         | Text          | The full content of the tweet.                                                              | 
| tweet_coord                  | Geospatial    | Latitude and longitude coordinates where the tweet was posted, if available.                | 
| tweet_created                | Datetime      | Timestamp when the tweet was created.                                                       | 
| tweet_location               | Text          | Location specified in the user's profile.                                                   | 
| user_timezone                | Categorical   | Time zone specified in the user's profile.                                                  | 

<br/>

## Data Used for Modeling

| Feature                      | Data Type   | Description  |
|-----------------------------|-------------|--------------|
| **Target Variable: `encoded_sentiment`** | Categorical | This is an engineered variable derived from `airline_sentiment` for multi-class sentiment classification. It encodes sentiment as: 0 = Negative, 1 = Neutral, 2 = Positive. |
| **Feature: `text`**         | Text        | Contains consumer tweets about U.S. airlines. This field undergoes preprocessing, including removal of URLs and mentions (`@`), and stopword removal |


<br/>

# 1. Load Data

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv("../../data/tweets.csv")

In [2]:
# Show basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [3]:
# Show the first few rows
df.head(3)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)


# 2. Data Preprocessing

## 2.1 Handle Duplicates

In [4]:
# Check for duplicate rows
duplicate_rows = df[df.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Drop duplicate rows
df.drop_duplicates(inplace=True)

Number of duplicate rows: 36


In [5]:
# Confirm the shape after removal
print(f"Shape after dropping duplicates: {df.shape}")

Shape after dropping duplicates: (14604, 15)


## 2.2 Handle Missing Values

In [6]:
# Check for missing values for each variables in the dataset
print("\nMissing values count for each variables:")
print("-------------------------------------------")
print(df.isnull().sum())

print("""\n\n**Note**: We won't remove any rows with missing values here as 
our main field we use is 'text' and 'airline_sentiment' column,
which has no missing values""")


Missing values count for each variables:
-------------------------------------------
tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5445
negativereason_confidence        4101
airline                             0
airline_sentiment_gold          14564
name                                0
negativereason_gold             14572
retweet_count                       0
text                                0
tweet_coord                     13589
tweet_created                       0
tweet_location                   4723
user_timezone                    4814
dtype: int64


**Note**: We won't remove any rows with missing values here as 
our main field we use is 'text' and 'airline_sentiment' column,
which has no missing values


## 2.3 Text Processing

In [7]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\tys\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tys\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tys\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### 2.3.1 Feature Engineering

For feature engineering in sentiment analysis, we will perform the following steps:

- `Stopwords Removal`: Eliminating common words (e.g., "the", "is", "and") that don't contribute meaningful information.

In [8]:
import re

# Initialize stopwords, stemmer
stop_words = set(stopwords.words('english'))

# Step 1: Lowercase and clean the text
def clean_text(text):
    text = text.lower()                                 # Lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text) # Remove URLs
    text = re.sub(r'@\w+', '', text)                    # Remove mentions
    # text = re.sub(r'#\w+', '', text)                    # NOTE: Do not remove hashtags, 
                                                                # as there is a lot of hashtags with sentiment indication, 
                                                                # such as '#thankyou', '#happycustomer', etc...
    # text = re.sub(r'[^a-z\s]', '', text)                # NOTE: no need to remove numbers and punctuation for llm embedding
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Copy only the 'text' column to df_copy
df2 = df[['text', 'airline_sentiment']].copy()

# processed text after rule-based removing URL, twitter username
df2['clean_text'] = df2['text'].apply(clean_text)



In [9]:
df2.shape

(14604, 3)

In [10]:
# Check for missing values per column
print("\nMissing values per column:")
print(df2.isnull().sum())


Missing values per column:
text                 0
airline_sentiment    0
clean_text           0
dtype: int64


### 2.3.2 Target Engineering

We will convert the 'airline_sentiment' column into numerical values to use it as the target variable in our model, where `negative` = 0, `neutral` = 1, and `positive` = 2 

In [11]:
print("The target variable contains unique values of: ", df2['airline_sentiment'].unique(), 
      "which we are going to map it into 0, 1 and 2 respectively")

# Encode the sentiment column
df2['encoded_sentiment'] = df2['airline_sentiment'].map({"negative": 0, "neutral": 1, "positive": 2})


The target variable contains unique values of:  ['neutral' 'positive' 'negative'] which we are going to map it into 0, 1 and 2 respectively


## 3.0 Data Preparation for Modeling

In [12]:
df2

,text,airline_sentiment,clean_text,encoded_sentiment
0,@VirginAmerica What @dhepburn said.,neutral,what said.,1
1,@VirginAmerica plus you've added commercials t...,positive,plus you've added commercials to the experienc...,2
2,@VirginAmerica I didn't today... Must mean I n...,neutral,i didn't today... must mean i need to take ano...,1
3,@VirginAmerica it's really aggressive to blast...,negative,"it's really aggressive to blast obnoxious ""ent...",0
4,@VirginAmerica and it's a really big bad thing...,negative,and it's a really big bad thing about it,0
...,...,...,...,...
14635,@AmericanAir thank you we got on a different f...,positive,thank you we got on a different flight to chic...,2
14636,@AmericanAir leaving over 20 minutes Late Flig...,negative,leaving over 20 minutes late flight. no warnin...,0
14637,@AmericanAir Please bring American Airlines to...,neutral,please bring american airlines to #blackberry10,1
14638,"@AmericanAir you have my money, you change my ...",negative,"you have my money, you change my flight, and d...",0


In [13]:
# Split into features (X) and target labels (y)
X = df2['clean_text']
y = df2['encoded_sentiment']

### 3.1 Train test split with stratified sampling

In [14]:
from sklearn.model_selection import train_test_split

# Split before SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                        stratify=y,            
                                        test_size=0.2, 
                                        random_state=42)

In [15]:
pd.concat([X_train, y_train], axis=1).to_csv("../../data/train.csv")
pd.concat([X_test, y_test], axis=1).to_csv("../../data/test.csv")

In [16]:
# Verify that class distribution is preserved after the train-test split (i.e., stratified correctly)

print("Train class distribution:")
print(y_train.value_counts(normalize=True))

print("\nTest class distribution:")
print(y_test.value_counts(normalize=True))

print("""\n**Observation**: The class proportions appear to be preserved across the training and test sets, 
indicating a successful stratified split.""")

Train class distribution:
encoded_sentiment
0    0.627151
1    0.211675
2    0.161174
Name: proportion, dtype: float64

Test class distribution:
encoded_sentiment
0    0.627182
1    0.211571
2    0.161246
Name: proportion, dtype: float64

**Observation**: The class proportions appear to be preserved across the training and test sets, 
indicating a successful stratified split.


### 3.2 Text Representation with LLM embedding

In [17]:
import os
import pandas as pd 
from utils.prepare_llm_embedding import generate_embeddings_from_series 

EMBEDDING_TRAIN = "../../data/llm_embedding_train.csv"
if os.path.exists(EMBEDDING_TRAIN):
    pass
else:
    processed_text_series = pd.Series(X_train.to_list(),
                                    index=X_train.index.to_list()) 
    llm_embedding = generate_embeddings_from_series(processed_text_series,
                            additional_data={"encoded_sentiment": y_train.to_list()},
                            output_csv_path="../../data/llm_embedding_train.csv",
                            max_workers=20)
    print(llm_embedding)

In [18]:
import pandas as pd 
from utils.prepare_llm_embedding import generate_embeddings_from_series 

EMBEDDING_TEST = "../../data/llm_embedding_test.csv"
if os.path.exists(EMBEDDING_TEST):
    pass
else:
    processed_text_series = pd.Series(X_test.to_list(), 
                                    index=X_test.index.to_list()) 
    llm_embedding = generate_embeddings_from_series(processed_text_series,
                            additional_data={"encoded_sentiment": y_test.to_list()},
                            output_csv_path="../../data/llm_embedding_test.csv",
                            max_workers=20) 
    print(llm_embedding)

In [19]:
import numpy
import json

train_vectorized = pd.read_csv("../../data/llm_embedding_train.csv")
test_vectorized = pd.read_csv("../../data/llm_embedding_test.csv")

X_train_vectorized = train_vectorized["embedding_json"].apply(json.loads) # convert string into a list of 765 items in 1 column
X_train_vectorized = numpy.vstack(X_train_vectorized) # turn that list of 765 items into 765 features / columns
y_train = train_vectorized["encoded_sentiment"]

X_test_vectorized = test_vectorized["embedding_json"].apply(json.loads)
X_test_vectorized = numpy.vstack(X_test_vectorized)
y_test = test_vectorized["encoded_sentiment"]

### 3.3 Handling class imbalance issue with SMOTE

In [20]:
from imblearn.over_sampling import SMOTE

# Handling imbalanced using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)

In [21]:
# Check class distribution before and after applying SMOTE to confirm successful balancing

print("Class distribution in training set (before SMOTE):")
print(y_train.value_counts())

print("\nClass distribution in training set (after SMOTE):")
print(y_train_resampled.value_counts())

print("""\n**Observation**: The class distribution in the training set has been balanced after applying SMOTE, 
confirming that oversampling was successful.""")


Class distribution in training set (before SMOTE):
encoded_sentiment
0    7327
1    2473
2    1883
Name: count, dtype: int64

Class distribution in training set (after SMOTE):
encoded_sentiment
0    7327
2    7327
1    7327
Name: count, dtype: int64

**Observation**: The class distribution in the training set has been balanced after applying SMOTE, 
confirming that oversampling was successful.


## 4.0 Modeling

(i) Logistic Regression

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
model.fit(X_train_resampled, y_train_resampled)

# Predictions on training and test sets
y_train_pred_lr = model.predict(X_train_resampled)
y_test_pred_lr = model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_lr)
test_accuracy = accuracy_score(y_test, y_test_pred_lr)

# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_lr))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_lr))

Train Accuracy: 0.8882
Test Accuracy:  0.8405

Classification Report (Test):
              precision    recall  f1-score   support

           0       0.95      0.86      0.90      1832
           1       0.65      0.80      0.72       618
           2       0.76      0.82      0.79       471

    accuracy                           0.84      2921
   macro avg       0.79      0.83      0.80      2921
weighted avg       0.86      0.84      0.85      2921

Confusion Matrix (Test):
[[1576  191   65]
 [  65  495   58]
 [  14   73  384]]


(ii) Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the Decision Tree
dt_model = DecisionTreeClassifier(max_depth=100, 
                                  min_samples_split=10, 
                                  criterion='entropy', 
                                  min_samples_leaf=100,
                                  random_state=42)

# Train the model
dt_model.fit(X_train_resampled, y_train_resampled)

# Predictions on training and test sets
y_train_pred_dt = dt_model.predict(X_train_resampled)
y_test_pred_dt = dt_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_dt)
test_accuracy = accuracy_score(y_test, y_test_pred_dt)

# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_dt))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_dt))

(iii) XGBoost

In [ ]:
import os
import joblib
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the XGBoost Classifier
model_path = "../../models/llm_embedding_xgb_model.pkl"
if os.path.exists(model_path):
    print("Loading existing model...")
    gb_model = joblib.load(model_path)
else:
    # Train the model
    xgb_model = xgb.XGBClassifier(max_depth=10,
                                random_state=42,
                                # Introduce randomness to make training faster and reduce overfitting
                                subsample=0.8, ## Uses 80% of the data for each tree.
                                colsample_bytree=0.8, ## Uses 80% of the features for each tree.
                                # the parameters below make the model trained faster by enabling parallelism
                                n_jobs = -1)
    xgb_model.fit(X_train_resampled, y_train_resampled)
    # Export and load previously trained model to avoid retraining every time
    joblib.dump(xgb_model, model_path)
    print("Model saved.")

# Predictions on training and test sets
y_train_pred_xgb = xgb_model.predict(X_train_resampled)
y_test_pred_xgb = xgb_model.predict(X_test_vectorized)

# Accuracy scores
train_accuracy = accuracy_score(y_train_resampled, y_train_pred_xgb)
test_accuracy = accuracy_score(y_test, y_test_pred_xgb)

# Output
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy:  {test_accuracy:.4f}\n")

print("Classification Report (Test):")
print(classification_report(y_test, y_test_pred_xgb))

print("Confusion Matrix (Test):")
print(confusion_matrix(y_test, y_test_pred_xgb))

## 5.0 Text Mining with Decision Tree plot and SHAP value analysis

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Plot decision tree
plt.figure(figsize=(200, 200))
plot_tree(dt_model, filled=True, max_depth=5, 
          feature_names=None, 
          class_names=["Negative", "Neutral", "Positive"])
plt.show()

- Reference: https://medium.com/nlplanet/two-minutes-nlp-explain-predictions-with-shap-values-2a0e34219177

In [ ]:
import shap

# Create a SHAP explainer
explainer = shap.Explainer(dt_model,
                           X_train_resampled, 
                           feature_names=None)

# Compute SHAP values for the test set
shap_values = explainer(X_test_vectorized)

print(shap_values.values.shape)


In [ ]:
# SHAP value analysis for one of positive-sentiment comments
ind = 3

original_text = pd.read_csv("../../data/test.csv")["clean_text"].iloc[ind]
processed_text = X_test.iloc[ind]

print("Original text: ", original_text)
print("Processed text: ", processed_text)
print("------------------------\n")

print(f"The original sentiment of the {ind}-rd text item is",
      y_test.replace({0: 'Negative', 1: 'Neutral', 2: 'Positive'}).iloc[ind])
print(f"The predicted sentiment of the {ind}-rd text item is",
      pd.Series(y_test_pred_dt).replace({0: 'Negative', 1: 'Neutral', 2: 'Positive'}).iloc[ind])
print("------------------------\n")

shap.initjs()
print("Probability of Negative Sentiment, f(x='Negative')")
shap.plots.waterfall(shap_values[ind,:,0])

print("Probability of Neutral Sentiment, f(x='Neutral)")
shap.plots.waterfall(shap_values[ind,:,1])

print("Probability of Positive Sentiment, f(x='Positive)")
shap.plots.waterfall(shap_values[ind,:,2])

In [ ]:
# Let’s do the same with a neutral review.

ind = 2

original_text = pd.read_csv("../../data/test.csv")["clean_text"].iloc[ind]
processed_text = X_test.iloc[ind]

print("Original text: ", original_text)
print("Processed text: ", processed_text)
print("------------------------\n")

print(f"The original sentiment of the {ind}-th row of text item is",
      y_test.replace({0: 'Negative', 1: 'Neutral', 2: 'Positive'}).iloc[ind])
print(f"The predicted sentiment of the {ind}-th row of text item is",
      pd.Series(y_test_pred_dt).replace({0: 'Negative', 1: 'Neutral', 2: 'Positive'}).iloc[ind])
print("------------------------\n")

shap.initjs()
print("Probability of Negative Sentiment, f(x='Negative')")
shap.plots.waterfall(shap_values[ind,:,0])

print("Probability of Neutral Sentiment, f(x='Neutral)")
shap.plots.waterfall(shap_values[ind,:,1])

print("Probability of Positive Sentiment, f(x='Positive)")
shap.plots.waterfall(shap_values[ind,:,2])

In [ ]:
# Let’s do the same with a negative review.

ind = 9

original_text = pd.read_csv("../../data/test.csv")["clean_text"].iloc[ind]
processed_text = X_test.iloc[ind]

print("Original text: ", original_text)
print("Processed text: ", processed_text)
print("------------------------\n")

print(f"The original sentiment of the {ind}-th row of text item is",
      y_test.replace({0: 'Negative', 1: 'Neutral', 2: 'Positive'}).iloc[ind])
print(f"The predicted sentiment of the {ind}-th row of text item is",
      pd.Series(y_test_pred_dt).replace({0: 'Negative', 1: 'Neutral', 2: 'Positive'}).iloc[ind])
print("------------------------\n")

shap.initjs()
print("Probability of Negative Sentiment, f(x='Negative')")
shap.plots.waterfall(shap_values[ind,:,0])

print("Probability of Neutral Sentiment, f(x='Neutral)")
shap.plots.waterfall(shap_values[ind,:,1])

print("Probability of Positive Sentiment, f(x='Positive)")
shap.plots.waterfall(shap_values[ind,:,2])